In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [3]:
df = pd.read_csv(r"C:\Users\lenovo\Desktop\verianalizi\users_errors.csv")

In [4]:
# Veri setinin ilk birkaç satırını ve bilgilerini kontrol et
print("Veri Seti İlk 5 Satır:")
print(df.head())
print("\nVeri Seti Bilgisi:")
print(df.info())
print("\nEksik Değerler:")
print(df.isnull().sum())

Veri Seti İlk 5 Satır:
   id  user_id       interaction_type                interaction_details  \
0   1        1          Invalid Input   Data value exceeds allowed range   
1   2        2          Invalid Input              Invalid email address   
2   3        3  Authentication Failed  Session timeout due to inactivity   
3   4        1             Data Error              Invalid email address   
4   5        2  Authentication Failed              Invalid email address   

  device_id  ip_address  location     interaction_time  
0   DEV1234  172.16.0.3     İzmir  2025-03-01 17:40:28  
1   DEV5678    10.0.0.2  İstanbul  2025-03-01 17:40:28  
2   DEV9999  172.16.0.3     İzmir  2025-03-01 17:40:28  
3   DEV5678  172.16.0.3  İstanbul  2025-03-01 17:50:52  
4   DEV9999    10.0.0.2     İzmir  2025-03-01 17:50:52  

Veri Seti Bilgisi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
--- 

In [5]:
# Eksik verileri düşür (veya doldur)
df = df.dropna()

In [6]:
# Kategorik sütunları tanımla
categorical_columns = ["interaction_type", "interaction_details", "device_id", "ip_address", "location"]

In [7]:
# Her sütun için ayrı LabelEncoder nesnesi oluştur ve sakla
label_encoders = {}
for col in categorical_columns:
    if col in df.columns:  # Sütunun veri setinde olduğundan emin ol
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col].astype(str))  # Her ihtimale karşı string'e çevir
        label_encoders[col] = encoder  # Encoder'ı sakla
        print(f"{col} sütunu için eşsiz değerler:", encoder.classes_)
    else:
        print(f"Uyarı: {col} sütunu veri setinde bulunamadı!")

interaction_type sütunu için eşsiz değerler: ['Authentication Failed' 'Data Error' 'Invalid Input' 'Permission Denied'
 'Session Expired']
interaction_details sütunu için eşsiz değerler: ['Data value exceeds allowed range' 'Incorrect password'
 'Invalid email address' 'Session timeout due to inactivity'
 'User not authorized to access resource']
device_id sütunu için eşsiz değerler: ['DEV1234' 'DEV5678' 'DEV9999']
ip_address sütunu için eşsiz değerler: ['10.0.0.2' '172.16.0.3' '192.168.1.1']
location sütunu için eşsiz değerler: ['Ankara' 'İstanbul' 'İzmir']


In [8]:
# interaction_time'ı sayısal hale çevir (Unix timestamp'e)
df['interaction_time'] = pd.to_datetime(df['interaction_time']).astype(np.int64) // 10**9  # Saniye cinsinden

In [9]:
# Dönüştürülen veriyi kontrol et
print("\nDönüştürülen Veri Seti İlk 5 Satır:")
print(df.head())


Dönüştürülen Veri Seti İlk 5 Satır:
   id  user_id  interaction_type  interaction_details  device_id  ip_address  \
0   1        1                 2                    0          0           1   
1   2        2                 2                    2          1           0   
2   3        3                 0                    3          2           1   
3   4        1                 1                    2          1           1   
4   5        2                 0                    2          2           0   

   location  interaction_time  
0         2        1740850828  
1         1        1740850828  
2         2        1740850828  
3         1        1740851452  
4         2        1740851452  


In [10]:
# Veriyi bir dosyaya kaydet (isteğe bağlı)
df.to_csv("C:/Users/lenovo/Desktop/verianalizi/users_errors_encoded.csv", index=False)
print("\nDönüştürülen veri 'users_errors_encoded.csv' dosyasına kaydedildi.")


Dönüştürülen veri 'users_errors_encoded.csv' dosyasına kaydedildi.


In [11]:
# Geri dönüşüm fonksiyonu (sayısal veriyi orijinal hale çevirme)
def decode_dataframe(df_encoded, label_encoders, time_column='interaction_time'):
    df_decoded = df_encoded.copy()
    for col, encoder in label_encoders.items():
        df_decoded[col] = encoder.inverse_transform(df_decoded[col])
    if time_column in df_decoded.columns:
        df_decoded[time_column] = pd.to_datetime(df_decoded[time_column], unit='s')
    return df_decoded
    

In [12]:
# Geri dönüştürülen veriyi kontrol et
df_decoded = decode_dataframe(df, label_encoders)
print("\nGeri Dönüştürülen Veri Seti İlk 5 Satır:")
print(df_decoded.head())


Geri Dönüştürülen Veri Seti İlk 5 Satır:
   id  user_id       interaction_type                interaction_details  \
0   1        1          Invalid Input   Data value exceeds allowed range   
1   2        2          Invalid Input              Invalid email address   
2   3        3  Authentication Failed  Session timeout due to inactivity   
3   4        1             Data Error              Invalid email address   
4   5        2  Authentication Failed              Invalid email address   

  device_id  ip_address  location    interaction_time  
0   DEV1234  172.16.0.3     İzmir 2025-03-01 17:40:28  
1   DEV5678    10.0.0.2  İstanbul 2025-03-01 17:40:28  
2   DEV9999  172.16.0.3     İzmir 2025-03-01 17:40:28  
3   DEV5678  172.16.0.3  İstanbul 2025-03-01 17:50:52  
4   DEV9999    10.0.0.2     İzmir 2025-03-01 17:50:52  


In [14]:
# Veriyi bir dosyaya kaydet (isteğe bağlı)
df.to_csv("C:/Users/lenovo/Desktop/verianalizi/users_errors_encoded.csv", index=False)
print("\nDönüştürülen veri 'users_errors_encoded.csv' dosyasına kaydedildi.")


Dönüştürülen veri 'users_errors_encoded.csv' dosyasına kaydedildi.
